In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 69.2 MB 16 kB/s eta 0:00:013
     |████████████████████████████████| 13.2 MB 147 kB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=8ca81b78ba2745f9dd61c102c031663400508d410213abd124b957ee5b8aff1a
  Stored in directory: /home/stu_13066019095/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import catboost as cbt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os

warnings.simplefilter('ignore')
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [3]:
seed = 2020

In [4]:
# !pip install lightgbm

In [5]:
df_feature = pd.read_pickle('data/feature.pkl')

In [6]:
df_feature.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDate_year,earliesCreditLine_year,issueDate_year_earliesCreditLine_year_minus,debt_ratio_year,applicationType_cnt,employmentLength_cnt,employmentTitle_cnt,grade_cnt,homeOwnership_cnt,initialListStatus_cnt,postCode_cnt,purpose_cnt,regionCode_cnt,subGrade_cnt,title_cnt,verificationStatus_cnt,applicationType_employmentLength_cnt,applicationType_employmentTitle_cnt,applicationType_grade_cnt,applicationType_homeOwnership_cnt,applicationType_initialListStatus_cnt,applicationType_postCode_cnt,applicationType_purpose_cnt,applicationType_regionCode_cnt,applicationType_subGrade_cnt,applicationType_title_cnt,applicationType_verificationStatus_cnt,employmentLength_applicationType_cnt,employmentLength_employmentTitle_cnt,employmentLength_grade_cnt,employmentLength_homeOwnership_cnt,employmentLength_initialListStatus_cnt,employmentLength_postCode_cnt,employmentLength_purpose_cnt,employmentLength_regionCode_cnt,employmentLength_subGrade_cnt,employmentLength_title_cnt,employmentLength_verificationStatus_cnt,employmentTitle_applicationType_cnt,employmentTitle_employmentLength_cnt,employmentTitle_grade_cnt,employmentTitle_homeOwnership_cnt,employmentTitle_initialListStatus_cnt,employmentTitle_postCode_cnt,employmentTitle_purpose_cnt,employmentTitle_regionCode_cnt,employmentTitle_subGrade_cnt,employmentTitle_title_cnt,employmentTitle_verificationStatus_cnt,grade_applicationType_cnt,grade_employmentLength_cnt,grade_employmentTitle_cnt,grade_homeOwnership_cnt,grade_initialListStatus_cnt,grade_postCode_cnt,grade_purpose_cnt,grade_regionCode_cnt,grade_subGrade_cnt,grade_title_cnt,grade_verificationStatus_cnt,homeOwnership_applicationType_cnt,homeOwnership_employmentLength_cnt,homeOwnership_employmentTitle_cnt,homeOwnership_grade_cnt,homeOwnership_initialListStatus_cnt,homeOwnership_postCode_cnt,homeOwnership_purpose_cnt,homeOwnership_regionCode_cnt,homeOwnership_subGrade_cnt,homeOwnership_title_cnt,homeOwnership_verificationStatus_cnt,initialListStatus_applicationType_cnt,initialListStatus_employmentLength_cnt,initialListStatus_employmentTitle_cnt,initialListStatus_grade_cnt,initialListStatus_homeOwnership_cnt,initialListStatus_postCode_cnt,initialListStatus_purpose_cnt,initialListStatus_regionCode_cnt,initialListStatus_subGrade_cnt,initialListStatus_title_cnt,initialListStatus_verificationStatus_cnt,postCode_applicationType_cnt,postCode_employmentLength_cnt,postCode_employmentTitle_cnt,postCode_grade_cnt,postCode_homeOwnership_cnt,postCode_initialListStatus_cnt,postCode_purpose_cnt,postCode_regionCode_cnt,postCode_subGrade_cnt,postCode_title_cnt,postCode_verificationStatus_cnt,purpose_applicationType_cnt,purpose_employmentLength_cnt,purpose_employmentTitle_cnt,purpose_grade_cnt,purpose_homeOwnership_cnt,purpose_initialListStatus_cnt,purpose_postCode_cnt,purpose_regionCode_cnt,purpose_subGrade_cnt,purpose_title_cnt,purpose_verificationStatus_cnt,regionCode_applicationType_cnt,regionCode_employmentLength_cnt,regionCode_employmentTitle_cnt,regionCode_grade_cnt,regionCode_homeOwnership_cnt,regionCode_initialListStatus_cnt,regionCode_postCode_cnt,regionCode_purpose_cnt,regionCode_subGrade_cnt,regionCode_title_cnt,regionCode_verificationStatus_cnt,subGrade_applicationType_cnt,subGrade_employmentLength_cnt,subGrade_employmentTitle_cnt,subGrade_grade_cnt,subGrade_homeOwnership_cnt,subGrade_initialListStatus_cnt,subGrade_postCode_cnt,subGrade_purpose_cnt,subGrade_regionCode_cnt,subGrade_title_cnt,subGrade_verificationStatus_cnt,title_applicationType_cnt,title_employmentLength_cnt,title_employmentTitle_cnt,title_grade_cnt,title_homeOwnership_cnt,title_initi

In [7]:
for f in df_feature.select_dtypes('object').columns:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

In [8]:
df_train = df_feature[df_feature.isDefault.notna()].copy()
df_test = df_feature[df_feature.isDefault.isna()].copy()

df_train.shape, df_test.shape

((800000, 338), (200000, 338))

In [9]:
ycol = 'isDefault'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], df_train.columns))

model = cbt.CatBoostClassifier(eval_metric='AUC',
                               max_depth=6,
                               learning_rate=0.05,
                               n_estimators=10000,
                               reg_lambda=12,
                               #                                task_type='GPU',
                               random_state=seed)

df_oof = df_train[['id', ycol]].copy()
df_oof['prob'] = 0
prediction = df_test[['id']]
prediction['prob'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(
        kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(
        fold_id + 1))

    cbt_model = model.fit(X_train,
                          Y_train,
                          eval_set=[(X_val, Y_val)],
                          verbose=100,
                          early_stopping_rounds=50)

    pred_val = cbt_model.predict_proba(
        X_val)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val

    pred_test = cbt_model.predict_proba(
        df_test[feature_names])[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': cbt_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del cbt_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

0:	test: 0.6914790	best: 0.6914790 (0)	total: 197ms	remaining: 32m 46s
100:	test: 0.7319181	best: 0.7319181 (100)	total: 10.7s	remaining: 17m 27s
200:	test: 0.7368289	best: 0.7368289 (200)	total: 21.7s	remaining: 17m 39s
300:	test: 0.7391781	best: 0.7391781 (300)	total: 31.8s	remaining: 17m 4s
400:	test: 0.7408381	best: 0.7408381 (400)	total: 42.3s	remaining: 16m 51s
500:	test: 0.7419473	best: 0.7419473 (500)	total: 52.9s	remaining: 16m 42s
600:	test: 0.7425542	best: 0.7425595 (598)	total: 1m 3s	remaining: 16m 30s
700:	test: 0.7430187	best: 0.7430187 (700)	total: 1m 13s	remaining: 16m 17s
800:	test: 0.7433221	best: 0.7433237 (799)	total: 1m 23s	remaining: 16m 2s
900:	test: 0.7435633	best: 0.7435679 (898)	total: 1m 33s	remaining: 15m 44s
1000:	test: 0.7437637	best: 0.7437637 (1000)	total: 1m 43s	remaining: 15m 34s
1100:	test: 0.7439577	best: 0.7439605 (1099)	total: 1m 54s	remaining: 15m 29s
1200:	test: 0.7440917	best: 0.7440975 (1197)	t

1600:	test: 0.7430276	best: 0.7430342 (1599)	total: 2m 48s	remaining: 14m 45s
1700:	test: 0.7430856	best: 0.7430913 (1687)	total: 2m 58s	remaining: 14m 30s
1800:	test: 0.7431835	best: 0.7431835 (1800)	total: 3m 8s	remaining: 14m 18s
1900:	test: 0.7432584	best: 0.7432625 (1898)	total: 3m 18s	remaining: 14m 6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.74326741
bestIteration = 1916

Shrink model to first 1917 iterations.


In [10]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby([
    'column'
])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,issueDate,6.565511
1,term,5.081582
2,revolBal,2.946453
3,dti,2.853208
4,debt_ratio_year,2.502862
5,purpose_subGrade_default_ratio,2.077918
6,annualIncome,2.001730
7,loanAmnt,1.665262
8,n14,1.577152
9,revolUtil,1.576035


In [11]:
auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('auc:', auc)

auc: 0.7434870090279698


In [12]:
os.makedirs('sub', exist_ok=True)

sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/jiedai_{}.csv'.format(auc), index=False)

In [13]:
os.makedirs('prob', exist_ok=True)

prediction.to_csv('prob/sub_cbt.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/oof_cbt.csv', index=False)

In [14]:
df_oof.head()

,id,isDefault,prob
0,1,0.0,0.240333
1,23,1.0,0.344511
2,28,0.0,0.120330
3,32,0.0,0.311702
4,42,1.0,0.295955


In [15]:
prediction.head()

,id,prob
800000,800000,0.054796
800001,800001,0.331217
800002,800002,0.339922
800003,800003,0.298638
800004,800004,0.388608
